In [18]:
import pandas as pd
import talib as ta
import glob
from datetime import datetime, date, time

In [42]:
files = glob.glob('J:\\RAW_data\\Currency\\futures\\*\\EURINR*.csv')

In [43]:
files

['J:\\RAW_data\\Currency\\futures\\21APR\\EURINR21APRFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21APR2022\\EURINR22APRFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21AUG\\EURINR21AUGFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21DEC\\EURINR21DECFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21FEB\\EURINR22FEBFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21JUL\\EURINR21JULFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21JUL2022\\EURINR22JULFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21JUN\\EURINR21JUNFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21JUN2022\\EURINR22JUNFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21MAR2022\\EURINR22MARFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21MAY\\EURINR21MAYFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21MAY2022\\EURINR22MAYFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21NOV\\EURINR21NOVFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21OCT\\EURINR21OCTFUT.csv',
 'J:\\RAW_data\\Currency\\futures\\21SEP\\EURINR21SEPFUT.csv']

In [ ]:
df = pd.read_csv()

In [ ]:
dragon = ta.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])

In [ ]:
df['dragon'] = dragon
df['RSI'] = ta.RSI(df['close'], timeperiod=14)

In [18]:
#GraveStone
flag = False
count = 0
pnl = []
for file in files:
    print("\n",file)
    df = pd.read_csv(file)
    dragon = ta.CDLGRAVESTONEDOJI(df['open'], df['high'], df['low'], df['close'])
    df['dragon'] = dragon
    df['RSI'] = ta.RSI(df['close'], timeperiod=14)
    df['SMA7'] = df['close'].rolling(7).mean()
    df['SMA14'] = df['close'].rolling(14).mean()
    x = file[-18:-12]
    print(x)
    df.to_csv("J:\\temp\\"+x+".csv")
    
    entry = 0
    for index,row in df.iterrows():
        if row['dragon']>0 and row['RSI'] > 70 and flag == False:
#             print('\n',row["close"])
            flag = True
            entry = row['close']
            print("\n Entry",entry)
            
        elif row['low'] < entry-0.01 and flag == True:
#         flag == True and count <15:
            print("Exit",row['close'])
#             print("Exit MA",row['SMA7'],row['SMA14'])
#             print(count)
            count = 0
            
#             print("pnl",entry-row['close'])
            pnl.append(entry-row['close'])
            entry = 0
            flag = False
            
        elif flag == True and count ==30:
            print("count", count, row['close'])
            flag = False
            pnl.append(entry-row['close'])
#             print("Exit MA",row['SMA7'],row['SMA14'])
#             print("pnl",entry-row['close'])
            count =0
            entry = 0
            
#             
            print("Exit count",row['close'],row['RSI'])
        elif count < 30:
            count = count +1

print(pnl)
        


 J:\RAW_data\\Currency\futures\21APR2022\EURINR22APRFUT.csv
EURINR

 Entry 84.535
count 30 84.55
Exit count 84.55 77.65545165722175

 Entry 84.5875
count 30 84.61
Exit count 84.61 83.63719718496205

 Entry 84.53
count 30 84.54
Exit count 84.54 74.262215943588

 Entry 83.09
count 30 83.0875
Exit count 83.0875 68.72924978809124

 Entry 83.1
Exit 83.085

 Entry 82.9875
count 30 82.99
Exit count 82.99 72.85889709225489

 Entry 82.895
count 30 82.91
Exit count 82.91 74.78446764873921

 Entry 82.915
Exit 82.885

 Entry 82.805
Exit 82.8025

 Entry 82.81
Exit 82.8025

 Entry 82.5925
count 30 82.5925
Exit count 82.5925 78.61790956630394

 Entry 82.9925
Exit 82.9925

 J:\RAW_data\\Currency\futures\21APR2022\EURUSD22APRFUT.csv
EURUSD

 J:\RAW_data\\Currency\futures\21APR2022\GBPINR22APRFUT.csv
GBPINR

 Entry 101.49
Exit 101.4825

 Entry 101.42
count 30 101.4225
Exit count 101.4225 70.81022475705358

 Entry 101.5425
count 30 101.545
Exit count 101.545 73.71635970056471

 Entry 101.55
Exit 101.532

In [44]:
#SL = low
#Target = 2*low

flag = False
count = 0

for file in files:
    pnl = []
    print("\n",file)
    df = pd.read_csv(file)
    dragon = ta.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
    df['dragon'] = dragon
    df['RSI'] = ta.RSI(df['close'], timeperiod=14)
    df['SMA7'] = df['close'].rolling(7).mean()
    df['SMA14'] = df['close'].rolling(14).mean()
    slowk, slowd = ta.STOCH(df['high'], df['low'], df['close'], fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df['slowk'] = slowk
    df['slowd'] = slowd
    df['aroon'] = ta.AROONOSC(df['high'], df['low'], timeperiod=14)

    
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    entry = 0
    sl =0
    target =0
    for index,row in df.iterrows():
        interest, waste = row['dateTime'].split(".")
        y = datetime.strptime(interest, '%Y-%m-%d %H:%M:%S')
        check = time(y.hour,y.minute,y.second)
        if row['dragon']>0 and row['aroon']<-50 and flag == False and time(10,30,0)<check<time(15,0,0):# and row['ADX']<20 and row['slowk']<30: 
#             print('\n',row["close"])
            flag = True
            entry = row['close']
            sl = min(0.02,row['close']-row['low'])
            target =2*sl
#             print(row['close'],row['high'],row['low'],row['open'])
#             print("\nEntry",entry, "ADX :",row['ADX'], "Aroon : ", row['aroon'])
#             print("Entry %k",row['slowk'], "%d :",row['slowd'],"RSI : ", row['RSI'])
#             print("Entry MA",row['SMA14']-row['SMA7'])
            
#         elif row['high'] > entry+0.01 and flag == True:
# #         flag == True and count <15:
# #             print("Exit normaL",row['close'],row['RSI'])
# #             print("Exit MA",row['SMA7'],row['SMA14'])
# #             print(count)
#             count = 0
            
# #             print("pnl",row['close']-entry)
#             pnl.append(row['close']-entry)
#             entry = 0
#             flag = False

            
        elif row['aroon']>50 and flag == True:
#             print("Exit SL",row['close'],"Aroon : ",row['aroon'])
#             print("count", count)
            flag = False
#             print("Count : ", count)
            pnl.append(row['close']-entry)
#             print("Exit MA",row['SMA7'],row['SMA14'])
#             print("pnl SL",row['close']-entry)
            count =0
            entry = 0
    
            
#         elif (entry-row['close'])<(entry-0.05) and flag == True:
#             print("Exit count",row['close'],row['RSI'])
# #             print("count", count)
#             flag = False
#             pnl.append(row['close']-entry)
# #             print("Exit MA",row['SMA7'],row['SMA14'])
#             print("pnl count",row['close']-entry)
#             count =0
#             entry = 0
        elif flag == True and time(4,0,0)==check:
#             print("Exit count",row['close'],row['RSI'])
#             print("count", count)
            flag = False
            pnl.append(row['close']-entry)
#             print("Exit MA",row['SMA7'],row['SMA14'])
#             print("pnl count",row['close']-entry)
            count =0
            entry = 0
            
#             
#             print("Exit count",row['close'],row['RSI'])
        elif count < 1000:
            count = count +1

#     print(pnl)
    print(sum(pnl))
        


 J:\RAW_data\Currency\futures\21APR\EURINR21APRFUT.csv
0.26750000000005514

 J:\RAW_data\Currency\futures\21APR2022\EURINR22APRFUT.csv
0.002500000000040359

 J:\RAW_data\Currency\futures\21AUG\EURINR21AUGFUT.csv
-0.017499999999955662

 J:\RAW_data\Currency\futures\21DEC\EURINR21DECFUT.csv
0.25499999999998124

 J:\RAW_data\Currency\futures\21FEB\EURINR22FEBFUT.csv
0.4075000000000557

 J:\RAW_data\Currency\futures\21JUL\EURINR21JULFUT.csv
0.31749999999996703

 J:\RAW_data\Currency\futures\21JUL2022\EURINR22JULFUT.csv
-0.36249999999998295

 J:\RAW_data\Currency\futures\21JUN\EURINR21JUNFUT.csv
-0.07999999999998408

 J:\RAW_data\Currency\futures\21JUN2022\EURINR22JUNFUT.csv
0.10499999999993292

 J:\RAW_data\Currency\futures\21MAR2022\EURINR22MARFUT.csv
0.24749999999997385

 J:\RAW_data\Currency\futures\21MAY\EURINR21MAYFUT.csv
0.08999999999994657

 J:\RAW_data\Currency\futures\21MAY2022\EURINR22MAYFUT.csv
0.20500000000004093

 J:\RAW_data\Currency\futures\21NOV\EURINR21NOVFUT.csv
-0.05750

In [25]:
su = []
for i in pnl:
    if i <1:
        su.append(i) 
print(sum(su))

-0.015000000000007674


In [101]:

df = pd.read_csv('J:\\RAW_data\\\\Commodity\\futures\\21APR2022\\GOLD22JUNFUT.csv')
dragon = ta.CDLDRAGONFLYDOJI(df['open'], df['high'], df['low'], df['close'])
df['dragon'] = dragon
df['RSI'] = ta.RSI(df['close'], timeperiod=14)
df['SMA7'] = df['close'].rolling(7).mean()
df['SMA14'] = df['close'].rolling(14).mean()

entry = 0
for index,row in df.iterrows():
    if row['dragon']>0 and row['RSI'] < 50 and flag == False:
#             print('\n',row["close"])
        flag = True
        entry = row['close']
        print("\n Entry",entry)

    elif row['close'] > entry+5 and flag == True:
#         flag == True and count <15:
        print("Exit",row['close'],row['RSI'])
        print("Exit MA",row['SMA7'],row['SMA14'])
        print(count)
        count = count+1

        print("pnl",row['close']-entry)
        pnl.append(row['close']-entry)
        entry = 0
        flag = False

    elif flag == True and count ==30:
        print("count", count)
        flag = False
        pnl.append(row['close']-entry)
        print("Exit MA",row['SMA7'],row['SMA14'])
        print("pnl",row['close']-entry)
        count =0
        entry = 0

#             
        print("Exit count",row['close'],row['RSI'])
    elif count < 30:
        count = count +1

print(pnl)
        

Exit 52122.0 nan


AttributeError: 'Series' object has no attribute 'ix'

In [102]:
for index,row in df.iterrows():
    df.ix[index]

AttributeError: 'DataFrame' object has no attribute 'ix'